<a href="https://colab.research.google.com/github/Raian-Rahman/Pytorch-Practice/blob/main/(CNN)Cifer10_Dataset_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import random_split,DataLoader

%matplotlib inline


### Loading the dataset

In [ ]:
train_data = datasets.CIFAR10('data',train=True,download='True',transform=transforms.ToTensor())
print(len(train_data))
train,val = random_split(train_data,[45000,5000])
train_loader = DataLoader(train,batch_size = 32)
val_loader = DataLoader(val,batch_size=32)

Extracting data/cifar-10-python.tar.gz to data
50000


# Defining the CNN Architecture

As it is a custom CNN so I am defining the architecture like below

Input -> CONV+MaxPOOL (5) -> CONV+MaxPOOL (5) -> CONV+MaxPOOL (5) -> CONV+MaxPOOL (3) -> FC -> FC -> Softmax

Initially the image size is 32 by 32

32x32x3 -> 28x28x8 -> 14x14x8 -> 10x10x16 -> 5x5x16 -> 4x4x32 -> 2x2x32

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()

        #convolutional layer
        self.conv0 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size= 5)
        self.conv1 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size= 5)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size= 2)

        #max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)


        #fully connected layer
        self.fc1 = nn.Linear(400,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,32)
        self.fc4 = nn.Linear(32,16)
        self.fc5 = nn.Linear(16,10)
        
    def forward(self,x):
        #conv with ReLU and Pooling 
        x = self.conv0(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv1(x)
        
        x = F.relu(x)
        x = self.pool(x)

        # x = self.conv2(x)
        # x = F.relu(x)
        # x = self.pool(x)
        #flattenned layer        
        x = x.view(-1,5*5*16)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return x
        
model = Net().cuda()
print(model)

Net(
  (conv0): Conv2d(3, 8, kernel_size=(5, 5), stride=(1, 1))
  (conv1): Conv2d(8, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (fc5): Linear(in_features=16, out_features=10, bias=True)
)


# Defining a Loss Function and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(),lr=0.01)
loss = nn.CrossEntropyLoss()

In [ ]:
# define training loop

nb_epochs = 25

for epoch in range(nb_epochs):
    losses = []
    for batch in train_loader:
        x ,y = batch
        # x : batch_size * 1_channel * 28 * 28
        #b = x.size(0)
        #x = x.view(b,-1).cuda()
        #forward pass
        x = x.cuda()
        l = model(x) # l stands for logit
        
        #compute the loss fucnction
        J = loss(l,y.cuda())
        
        #cleaning the gradient 
        optimiser.zero_grad()
        #params.grad._zero()
       
        #compute accumulatinfg the partial gradient 
        J.backward() #params.
        #params.grad.add_(dJ/dparams)
        
        #update the parameters
        optimiser.step()
        #with torch.no_grad(): params = params - alpha*params.grad
        losses.append(J.item())
    print(f'Epoch {epoch+1},train loss: {torch.tensor(losses).mean():.2f}',sep = ' ')
    losses = []
    accuracies = []
    for batch in val_loader:
        x ,y = batch
        # x : batch_size * 1_channel * 28 * 28
        x = x.cuda()
        #forward pass
        with torch.no_grad():
            l = model(x) # l stands for logit
            #compute the loss fucnction
            J = loss(l,y.cuda())
            losses.append(J.item())
            accuracies.append(y.cuda().eq(l.detach().argmax(dim=1)).float().mean())
    print(f' validation loss: {torch.tensor(losses).mean():.2f}')
    print(f'Epoch {epoch+26},Accuracy: {torch.tensor(accuracies).mean():.5f}')

Epoch 1,train loss: 0.89
 validation loss: 1.51
Epoch 26,Accuracy: 0.56150
Epoch 2,train loss: 0.88
 validation loss: 1.55
Epoch 27,Accuracy: 0.56369
Epoch 3,train loss: 0.86
 validation loss: 1.56
Epoch 28,Accuracy: 0.56648
Epoch 4,train loss: 0.85
 validation loss: 1.58
Epoch 29,Accuracy: 0.55354
Epoch 5,train loss: 0.83
 validation loss: 1.59
Epoch 30,Accuracy: 0.56270
Epoch 6,train loss: 0.82
 validation loss: 1.74
Epoch 31,Accuracy: 0.54578
Epoch 7,train loss: 0.81
 validation loss: 1.67
Epoch 32,Accuracy: 0.55732
Epoch 8,train loss: 0.80
 validation loss: 1.72
Epoch 33,Accuracy: 0.55673
Epoch 9,train loss: 0.79
 validation loss: 1.74
Epoch 34,Accuracy: 0.55275
Epoch 10,train loss: 0.78
 validation loss: 1.71
Epoch 35,Accuracy: 0.55693
Epoch 11,train loss: 0.77
 validation loss: 1.73
Epoch 36,Accuracy: 0.56469
Epoch 12,train loss: 0.76
 validation loss: 1.74
Epoch 37,Accuracy: 0.55951
Epoch 13,train loss: 0.76
 validation loss: 1.84
Epoch 38,Accuracy: 0.55175
Epoch 14,train loss: 